In [ ]:
%%bash

mkdir -p data/raw
curl -s https://www.dcc.fc.up.pt/~ines/MIMIC-III/ -o DB.html

grep -oP 'href="\K[^"]+\.csv\.gz' DB.html > DB_FILE_LIST.txt

while IFS= read -r FILE; do
    echo "Downloading: $FILE"
    curl -s -O "https://www.dcc.fc.up.pt/~ines/MIMIC-III/${FILE}"
    
    echo "Decompressing: $FILE"
    gunzip -f "$FILE"

    # Rimuove l'estensione .gz dal nome
    CSV_NAME="${FILE%.gz}"

    echo "Moving $CSV_NAME to data/raw/"
    mv "$CSV_NAME" data/raw/

done < DB_FILE_LIST.txt

rm -f DB.html DB_FILE_LIST.txt
